In [ ]:
import sympy.physics.mechanics as me
import sympy as sm
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import time

from scipy.linalg import solve_continuous_are, det, expm

This creates a decorator to test functions for usage of CPU time, line by line.\
To see the results, this line: *profiler.print_stats()* must be added. 

In [ ]:
from line_profiler import LineProfiler

profiler = LineProfiler()

def profile(func):
    def inner(*args, **kwargs):
        profiler.add_function(func)
        profiler.enable_by_count()
        return func(*args, **kwargs)
    return inner

This is a **3D n body pendulum**\
I put it into a vertical position, linearize the equations of motion and try to stabilize in by
- moving the point $P_{01}$, it is attached to the horizontal plane
- adding a torque to every body. (This is needed, because I model the pendulum bodies as RigidBody, and their rotation cannot be stopped by moving around the point of attachment)

The control variables are
- $F_x, F_z$ forces applied to the point $P_0$ where the pendulum is attached to the plane.
- Torques applied to each body fixed frame

The 'suspension point' is modeled as a Particle, rest are RigidBody\
(I basically copied this program from some other program quite some time ago, and only enlarged it to 3D and rigid bodies)\
For some reason, which I did not investigate, it only works well for $n \in \{{ 1, 3}\} $. Other values give problems.

I do these calculations:
- I calculate the full system without controls
- I linearize to get the optimum feed back, and use this feed back to numerically solve the full EOMs
- I numerically solve the linearized equations, using the optimum feed back
- A linear system $\dfrac{d}{dt} \hat y = RHS \cdot \hat y$, with RHS = const., has the closed form solution $\hat y = e^{\displaystyle{RHS \cdot t}} \cdot \hat y_0$, and I use scypi's *expm(..)* function to solve it.

**NOTE**:\
At first, I rotated the body fixed frames as 'xyz'. This gave problems with n = 2 bodies. The I changed the rotation to 'yxz'. This seemed to solve the problem with n = 2 and n = 5.\
I think, it is related the the *gimbal lock*, describerd here: https://moorepants.github.io/learn-multibody-dynamics/angular.html?highlight=lock (you have to scroll down a bit to find it) but not really clear, why.

In [ ]:
#===============================================
n =  3                  #number of pendulums (bodies)
plot_NL = True         # if True, the running of the note book is stopped after the full EOMs with controlls have been calculated, so this result may be animated.
#===============================================

start = time.time()    

N = me.ReferenceFrame('N')
P0 = me.Point('P0')
P0.set_vel(N, 0)                                      

# Generalized coordinates and velocities
q = []
u = []

A = [N]

Torque = []
for i in range(1, n+1):                                             # Data for pendulun i at at position [i-1]
    for j in ('x', 'y', 'z'):
        q.append(me.dynamicsymbols('q'+str(i) + str(j)))            #Rotation of frame i relative to N
        u.append(me.dynamicsymbols('u'+str(i) + str(j)))           
    A.append(me.ReferenceFrame('A'+ str(i)))
    Torque.append(me.dynamicsymbols('T' + str(i-1)))
        
q0x, q0z = me.dynamicsymbols('q0x q0z')                             # coordinates of attachment poi nt
u0x, u0z = me.dynamicsymbols('u0x, u0z')

g, l, m, c1, t = sm.symbols('g l m c1, t')                          # all masses, distances, time, friction are the same for all pendulums
r1 = sm.symbols('r1')                                               # distance from Pi to Dmc
iXX, iYY, iZZ = sm.symbols('iXX, iYY, iZZ')                         # moments of inertia around the center of mass, principal axes

Fx, Fz = me.dynamicsymbols('Fx, Fz')                                       #Control parameters. forces at attachment point acting on P01

P01 = P0.locatenew('P01', q0x*N.x + q0z*N.z)
P01.set_vel(N, u0x*N.x + u0z*N.z)

#Frame points, particles for each Pendulum, numbered 1, 2, ..., n
rot = [0]                                                           # rot, rot1 needed for the kinematical differential equations
rot1 = [0]
P = [P01]
DMC = [P01]                                                         # list of center of gravity of each body
BODY = [me.Particle('P01a', P01, m)]                                # listof bodies

# NOTE, see the comment above about gimbal lock
for i in range(1, n+1):
    A[i].orient_body_fixed(N, (q[3*i-2], q[3*i-3], q[3*i-1]), '213')  
    rot.append(A[i].ang_vel_in(N))
    A[i].set_ang_vel(N, u[3*i-3] * A[i].x + u[3*i-2] * A[i].y + u[3*i-1] * A[i].z )
    rot1.append(A[i].ang_vel_in(N))

    Pi = P[i-1].locatenew('P' + str(i), l * A[i].y)                 # old way to determine points
    Pi.v2pt_theory(P[i-1], N, A[i])
    P.append(Pi)
    Dmci = P[i-1].locatenew('Dmc' + str(i), r1 * A[i].y)            # location of center of gravity
    Dmci.v2pt_theory(P[i-1], N, A[i])
    DMC.append(Dmci)

    I = me.inertia(A[i], iXX, iYY, iZZ)
    Bodyi = me.RigidBody('Body' + str(i), Dmci, A[i], m, (I, Dmci))
    BODY.append(Bodyi)   

Set up **Kane's equations**

In [ ]:
# Kanes forrmalism
start1 = time.time()

#Kinematic equations
kd = [(q0x.diff(t) - u0x)] + [(q0z.diff(t) - u0z)] + [me.dot(rot[i] - rot1[i], uv) for uv in N for i in range(1, n+1)]

# forces, torques
FL = [(P01, -m*g*N.y + Fx*N.x + Fz*N.z)]

for i in range(1, n+1):
    FL.append((DMC[i], -m*g*N.y))
    FL.append((A[i], Torque[i-1]*A[i].y))

q_ind = [q0x, q0z] + q
u_ind = [u0x, u0z] + u

KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
print('MM DS', me.find_dynamicsymbols(MM))
print('MM FS', MM.free_symbols)
print(f'MM contains {sm.count_ops(MM):,} operations, {sm.count_ops(sm.cse(MM)):,} after cse, \n')

force = KM.forcing_full
print('force DS', me.find_dynamicsymbols(force))
print('force FS', force.free_symbols)
print(f'force contains {sm.count_ops(force):,} operations, {sm.count_ops(sm.cse(force)):,} after cse \n')
print("it took {:.3f} sec to establish Kane's equations".format(time.time() - start1))

**Lambdification**

In [ ]:
# Carthesian coordinates of the center points, needed only for plotting them
orte_Dmc = ([ [me.dot(ort.pos_from(P0), uv) for uv in N] for ort in DMC] )

qL = q_ind + u_ind
pL = [g, m, l, r1, iXX, iYY, iZZ, c1, Fx, Fz] + Torque                                     

MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

orte_Dmc_lam = []
for ort in orte_Dmc:
    orte_Dmc_lam.append(sm.lambdify(qL + pL, ort))

**Numerical Integration**\
Normally in my simulations I explicitly set all parameters and initial conditions.\
As here I am after something else, linearization and control, I simply pre_set them.\
Of course they may be changed, just a bit more complicated.\
This is the uncontrolled motion, below I linearize around the vertical position and control it.

In [ ]:
start2 = time.time()
intervall = 20
schritte = 250
times = np.linspace(0, intervall, schritte)

#Anfangsbedingungen
#qL = [q0x, q0z] + q + [u0x, u0z] + u
#pL = [g, m, l, r1, iXX, iYY, iZZ, c1, Fx, Fz] + Torque    
                                  
pL_vals = [9.8, 1, 10, 5., 1/2, 1/3, 1/3, 0., 0., 0.] + [ 0. for _ in range(n)]
y0 = [0., 0.] + [0, 0, 0] * int(len(q)/3) + [0., 0.] + [0.1, np.random.choice((-1, 1)) * 0.1, 0.1] * int(len(u)/3)
                                            
def gradient(t, y, args):                        
    vals = np.concatenate((y, args))
    sol = np.linalg.solve(MM_lam(*vals), force_lam(*vals))
    return np.array(sol).T[0]

times = np.linspace(0., intervall, int(schritte*intervall))
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), atol=1.e-6, rtol=1.e-6) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start2):.3f} CPU - sec  ')

Plot the results

In [ ]:
bez = ['X', 'Y', 'Z']
fig,(ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

x_coords = np.zeros((len(times), n+1))
y_coords = np.zeros((len(times), n+1))
z_coords = np.zeros((len(times), n+1))

for i in range(len(times)):
    for k, ort in enumerate(orte_Dmc_lam):
        x_coords[i, k] = ort(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[0]
        y_coords[i, k] = ort(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[1]
        z_coords[i, k] = ort(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[2]
    
for j in range(0, n+1):
    ax2.plot(times, x_coords[:,j], label='X-Koordinate von Punkt {}'.format(j))
    ax2.plot(times, y_coords[:,j], label='Y-Koordinate von Punkt {}'.format(j))
    ax2.plot(times, z_coords[:,j], label='Z-Koordinate von Punkt {}'.format(j))
ax2.set_title('Coordinates of center of gravity')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Coordinates [m]')
ax2.legend(loc='upper right');

**Linearisation**

I use sympy's method as expalined here: https://docs.sympy.org/latest/modules/physics/mechanics/linearize.html

Notes:
- the op_point dictionary *must* also include the values of the *accelerations* of the coordinates. This is not very obvious from the documentatrion, JM explained this to me.
- the controls variable must be *dynamic symbols*. This is clearly explained in the documentation. Here, they are $F_x, F_z, T_i$ and they are declared to be dynamic symbols above.

This is much faster than trying to do it 'from first principles' using jacobian operators - as I did it in my related example.


In [ ]:
start3 = time.time()
dict_eq = {i: 0. for i in qL} | {i.diff(t): 0. for i in u_ind}

F_A, F_B, input_vector = KM.linearize(A_and_B=True, op_point=dict_eq, new_method=True)
print('input_vector', input_vector)                                                                     # input vector contains the control parameters
print('F_A DS', me.find_dynamicsymbols(F_A))
print('F_A FS', F_A.free_symbols)
print(f'F_A contains {sm.count_ops(F_A):,} operations, {sm.count_ops(sm.cse(F_A)):,} after cse \n')

print('F_B', me.find_dynamicsymbols(F_B))
print('F_B FS', F_B.free_symbols)   
print(f'F_B contains {sm.count_ops(F_B):,} operations, {sm.count_ops(sm.cse(F_B)):,} after cse \n')

print(f'It took {(time.time() - start3):.3f} sec to linearize the EOMs')

This funktion integrates the 'dynamic Riccati equation', equation (9.18) of this book:\
https://www.amazon.de/Mathematical-Control-Theory-Introduction-Applications/dp/3030447766/ref=sr_1_1?__mk_de_DE=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=31QL7QCEXT61I&keywords=mathematical+control+theory&qid=1705340852&sprefix=mathematical+control+theory%2Caps%2C106&sr=8-1

In [ ]:
#@profile
def integrate_riccati(A, B, Q, R, end_zeit=10, schritte=500, method='RK45', **kwargs):
    '''
    This solves the Riccati equation, eq (9.18) of this book
    https://www.amazon.de/Mathematical-Control-Theory-Introduction-Applications/dp/3030447766/ref=sr_1_1?__mk_de_DE=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=31QL7QCEXT61I&keywords=mathematical+control+theory&qid=1705340852&sprefix=mathematical+control+theory%2Caps%2C106&sr=8-1
    
    d/dtP = Q + P A + A.T P - P B R^-1 B.T P, where I (arbitrarily) take P(0) = I
    I integrate  from 0. to end_zeit,
    schritte * end_zeit gives the number of integration steps to be returned 
    '''
    n = Q.shape[0]
    m = Q.shape[1]

    P0 = np.identity(n).flatten()

    def gradient(t, P0):
        P0 = P0.reshape(n, m)           
        RHS = (Q + P0 @ A + A.T @ P0 - P0 @ B @ np.linalg.inv(R) @ B.T @ P0).flatten()
        P0 = P0.flatten()
        return RHS

    times = np.linspace(0., end_zeit, int(schritte*end_zeit))
    t_span = (0., end_zeit)

    resultat1 = solve_ivp(gradient, t_span, P0, t_eval = times, method=method, options=kwargs)
    resultat = resultat1.y.T
    print(resultat1.message)
    resultat = resultat.reshape(len(resultat), n, m)
    return resultat

The linearized sytem is: $\dfrac{d}{dt} \bar y = F_A \cdot \bar y + F_B \cdot \bar u$, where $\bar u$ is the control, and $F_A, F_B$ are the matrices obtained above.

Calculate the **optimal feed back** to stabilize the linearized system.\
Let P be the solution to the Riccati equation $Q + P \cdot F_A + F_A^T \cdot P - P \cdot B \cdot R^{-1} \cdot F_B^T \cdot P = 0$, where $Q, R$ are the weight matrices in the cost functional related to the coordinates and the control, respectively.\
Then $\hat{\overline{control}} = \left[ R^{-1} \cdot F_B^T \cdot P \right] \cdot \bar y$ ist the optimal control.\
For lack of better reason, I pick $R$ and $Q$ to be the unit matrices of appropriate dimensions.\
The scipy function **solve_continuous_are** numerically solves the *Ricatti equation*.\
Controllability means, I think, that any starting configuration of the system may be moved to another predetermined position via the controls. Intuitively clear, that with this pendulum this is only the case for n = 1\
All of this is in books about (linear) control theory. 

In [ ]:
start4 = time.time()

pL = [g, m, l, r1, iXX, iYY, iZZ, c1, Fx, Fz] + Torque
pL_vals = [9.8, 1., 10., 5., 1, 2, 3, 1.,  0, 0] + [0. for i in range(len(Torque))]

A_lam = sm.lambdify(pL, F_A, cse=True)
B_lam = sm.lambdify(pL, F_B, cse=True)

Alin = A_lam(*pL_vals)
Blin = B_lam(*pL_vals)

print('determinant of Alin', det(Alin))
#Is the system controllable? This seems to be only a sufficient condition (?)
def controllable(a, b):
    a = np.matrix(a)
    b = np.matrix(b)
    n = a.shape[0]
    controllable_matrix = []
    for i in range(n):
        controllable_matrix.append(a ** i * b)
    controllable_matrix = np.hstack(controllable_matrix)
    return np.linalg.matrix_rank(controllable_matrix) == n
print('System controllable?  ', controllable(Alin, Blin))

#compute optimal gains K
Q = np.eye(Alin.shape[1])
R = np.eye(Blin.shape[1])
S = solve_continuous_are(Alin, Blin, Q, R) #solves Riccati equation
K = np.dot(np.dot(np.linalg.inv(R), Blin.T), S)

print('it took {:.3f} sec to calculate the feed back'.format(time.time() - start4))

Her I just want to see how close the solution of the integratred Riccati equation is to the solution of the algebraic Riccati equation, solved above.\
As the norm of the matrix, I use the largest entry of it: $ || M || := max_{i, j} |M_{ij}|$.\
At least here, it seems to converge quite rapidly.\
Just a side line.

In [ ]:
end_zeit = 100.
TEST = integrate_riccati(Alin, Blin, Q, R, end_zeit=end_zeit, schritte=500, kwargs={'rtol': 1.e-8, 'rtol': 1.3-8})
fehler = np.max(np.abs(np.subtract(TEST[-1], S))) / np.max(np.abs(S)) * 100
print(f'The maximal error of the integrated Riccati - ARE is {fehler:.3f} % of max entry of S, when it was integrated from 0 to {end_zeit} sec, and the last solution used \n')
#profiler.print_stats()

**Numerical integration** of the **full system** using the optimum controls\
The optimal feed back is calculated each step of the integration.

**Important note**\
Only integrating this cell will tell you whether the system, given the initial disturbance, will actually be stabilized.\
Calculating the linearized versions, by definition!, will always look like you can control it.\
(If this integration runs more than a few seconds, it is likely not stabilizeable)

In [ ]:
pL1_vals = [9.8, 1., 10., 5.,  1., 2., 3., 1]
equilibrium_point = np.zeros(len(qL))

aa = []
for i in range(int(len(u)/3)):
    aa.append(0.0 * np.random.choice((-1., 1.)))
    aa.append(0.15 * np.random.choice((-1., 1.)))
    aa.append(0.)
y0 = [0., 0.] + [0.145, 0., 0.195] * int(len(q)/3) + [0., 0.] + aa 
y_store = list(y0)                                                      # store the initial conditions, so I do the solution using matrix exponentials below with the same initial conditions


start5 = time.time()

# The controls, which originally were in pL_vals will now, of course be the optimal feed back, and added during each step of the integration
# now: kraft = K @ (x(equilibrium) - x(t)). I store the forces and the times in lists, so that I can plot them later.
kraefte = []
zeiten  = []

#@profile
def gradient(t, x, args):
# calculate the control force
    kraft = np.dot(K, equilibrium_point - x)
    kraefte.append(kraft)
    zeiten.append(t)
    arguments = np.hstack((x, args, kraft))
    return  np.array(np.linalg.solve(MM_lam(*arguments), force_lam(*arguments))).T[0]

times = np.linspace(0., intervall, int(schritte*intervall)) 
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL1_vals,), atol=1.e-8, rtol=1.e-8) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start5):.3f} CPU - sec  ')
#profiler.print_stats()

Plot the results\
It can easily be seen, that the system is driven to its equilibrium point, if the initial disturbances are not too great.\
Note, that the results are a bit different from those obtained by solving the linearized EOMs below. I do not know why.\
**Note**: if you set **plot_NL = True** above, the program will be interrupted here, so the results obtained here wil not be overriden by the results of the subsequent calculations. So, you should start ther animation at the bottom of this program to start the animation.

In [ ]:
bez = ['X', 'Y', 'Z']
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles, integrating full EOMs, with controls')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

kraefte = np.array(kraefte)
for i, j in enumerate(('Fx', 'Fy', *['Torque on body ' + str(kk) for kk in range(1, n+1)])):
    ax2.plot(zeiten, kraefte[:, i], label=j)
ax2.set_title('Control forces / torques')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N] / Torque [Nm]')   
ax2.legend();

if plot_NL:
    raise Exception('go down to the last cell, the animation, and start it')

**Numerically integrate** the **linearized system**.

The sytem here is: $\dfrac{d}{dt} \bar y = F_A \cdot \bar y + F_B \cdot \bar u$, where $\bar u$ is the control.

Form the matrix $RHS$ to be calculated
 as follows:

First, form new state vektor $\hat y = \left[ \bar y, \bar u \right]$

The new rhs of the system is $RHS = \begin{bmatrix} F_A & F_B \\ 0_1 & 0_2 \end{bmatrix}$, where $0_i = \begin{bmatrix} 0. &....& 0. \\. &...&.\\ 0. &...& 0. \end{bmatrix}$ of appropriate dimensions.

Now I integrate the system numerically.\
( Of course, I could have done it the same way I did it with the full EOMs with the controls as parameters, I simply wanted to try another way.) 


In [ ]:
start6 = time.time()

RHS = sm.Matrix.hstack(F_A, F_B)
RHS = sm.Matrix.vstack(RHS, sm.zeros(F_B.shape[1], RHS.shape[1]))
print('RHS shape', RHS.shape)
print('RHS DS', me.find_dynamicsymbols(RHS))
print('RHS FS', RHS.free_symbols)
print(f'RHS contains {sm.count_ops(RHS):,} operations, {sm.count_ops(sm.cse(RHS)):,} after cse \n')

pL1 = [g, m, l, r1, iXX, iYY, iZZ, c1]
p1L_vals = [9.8, 1., 10., 5., 1, 2, 3, 1.]

# convert RHS from a sympy matrix to a numpy array
RHS_lam = sm.lambdify(pL1, RHS, cse=True)
RHS = RHS_lam(*pL1_vals)

# extend the equilibrium point to include the control forces
equilibrium_point1 = np.hstack((equilibrium_point, np.zeros(Blin.shape[1])))

# form y_hat, the initial conditions for the solution of the linearized system
y0 = np.array(list(y_store) + [0. for _ in range(Blin.shape[1])])      

# solve the linearized system, using matrix exponentials
resultat = []
kraefte  = []
zeiten   = [] 

#@profile
def gradient(t, y):
# get the optium control
    kraft = K @ (equilibrium_point - y[: len(qL)])
    kraefte.append(kraft)
    zeiten.append(t)
    y[len(qL):] = kraft
    return RHS @ (y - equilibrium_point1)

times = np.linspace(0., intervall, int(schritte*intervall))
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, atol=1.e-10, rtol=1.e-10) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start6):.3f} CPU - sec  ')
#profiler.print_stats()

Plot the results

In [ ]:
bez = ['X', 'Y', 'Z']
resultat_hilfs = []
for i in range(len(times)):
    resultat_hilfs.append(resultat[i, : len(qL)])
resultat = np.array(resultat_hilfs)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles, integrating the linearized system')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

kraefte = np.array(kraefte)
for i, j in enumerate(('Fx', 'Fy', *['Torque on body ' + str(kk) for kk in range(1, n+1)])):
    ax2.plot(zeiten, kraefte[:, i], label=j)
ax2.set_title('Control forces / torques')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N] / Torque [Nm]')   
ax2.legend();

Here I use scipy's **matrix exponential** solver *expm(..)* to solve the differential equation without integrating.\
I use the fact, that this idfferential equation $\dfrac{d}{dt} \hat y = RHS \cdot \hat y$, with RHS = const., has the solution $\hat y = e^{\displaystyle{RHS \cdot t}} \cdot \hat y_0$\
I have to to it in steps, using the result of the previous step as input for the next step, otherwise of course the controls will have no effect.\
At least with this example, this is the *fastest* way of doing it.

In [ ]:
start7 = time.time()
y = np.array(list(y_store) + [0. for _ in range(Blin.shape[1])])      
# solve the linearized system, using matrix exponentials
resultat = []
kraefte  = []
zeiten   = [] 

for i in range(len(times)-1):
# get the optium control
    kraft = K @ (equilibrium_point - y[: len(qL)])
    kraefte.append(kraft)
    zeiten.append(times[i])
    y[len(qL):] = kraft
# Here I do the stepwise solution.
    y = expm(RHS * (times[i+1] - times[i])) @ y
    resultat.append((y))
    
print(f'to calculate {intervall:.3f} sec took {(time.time() - start7):.3f} CPU - sec  ')

Plot the results

In [ ]:
resultat_hilfs = []
resultat = np.array(resultat)
for i in range(len(times)-1):
    resultat_hilfs.append(resultat[i, : len(qL)])
resultat = np.array(resultat_hilfs)
times = times[:-1]
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles, used matrix exponential')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

kraefte = np.array(kraefte)
for i, j in enumerate(('Fx', 'Fy', *['Torque on body ' + str(kk) for kk in range(1, n+1)])):
    ax2.plot(zeiten, kraefte[:, i], label=j)
ax2.set_title('Control forces / torques')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N] / Torque [Nm]')   
ax2.legend();
print(f'running the complete program takes {(time.time() - start):.3f} CPU - sec')

**Animation**

In my model, gravitation points in the negative Y direction, in matplotlib 3D, gravitation points in the negative Z direction.\
Hence, I must make this transformation:
- $N.x_{old} \rightarrow N.y_{new}$
- $N.y_{old} \rightarrow N.z_{new}$
- $N.z_{old} \rightarrow N.x_{new}$

The 'skeleton' of this program is from chat GPT

matplotlib seems to have no intrinsic object *sphere*. Trying to do it from 'first principles' takes VERY long.\
Hence, I marked the bodies simply with large markers.\
I left the spheres in the code, if anyone wants to test it.

Please note, that the *last* calculation is plotted.


In [ ]:
# reduce the number of points of time to around zeitpunkte
#=======================
zeitpunkte = 500
#=======================

# plot the horizontal plane H at Z = 0.
def plot_3d_plane(x_min, x_max, y_min, y_max):
    # Create a meshgrid for x and y values
    x = np.linspace(x_min, x_max, 100)
    y = np.linspace(y_min, y_max, 100)
    x, y = np.meshgrid(x, y)
    # Z values are set to 0 for the plane
    z = np.zeros_like(x)
    # Plot the 3D plane
    ax.plot_surface(x, y, z, alpha=0.1, rstride=100, cstride=100, color='c')

'''
# draw a shere in matplotlib 3D
def kugel(ax, radius, mittelpunkt, farbe):
    u = np.linspace(0, 2 * np.pi, 50)
    v = np.linspace(0, np.pi, 50)
    x = mittelpunkt[0] + radius * np.outer(np.cos(u), np.sin(v))
    y = mittelpunkt[1] + radius * np.outer(np.sin(u), np.sin(v))
    z = mittelpunkt[2] + radius * np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, color=farbe)
'''
times2 = []
resultat2 = []

reduction = max(1, int(len(times)/zeitpunkte))
for i in range(len(times)):
    if i % reduction == 0:
        times2.append(times[i])
        resultat2. append(resultat[i, 0: len(qL)])
  
schritte2 = len(times2)
print(f'animation used {schritte2} points in time')
resultat2 = np.array(resultat2)
times2 = np.array(times2)  

'''
# I assume, the pendulum bodies are sheres. I put a point on each one, so the rotation may be seen more easily.
rb  = sm.symbols('rb')     
rb1 = 2.5                                          # radius of the spheres
OBS = []
for i in range(1, n+1):
    OBS.append(me.Point('obs' + str(i)))
    OBS[i-1].set_pos(DMC[i], rb * A[i].x)
    OBS[i-1].v2pt_theory(DMC[i], N, A[i])
'''
trans = [N.z, N.x, N.y]                                         # aligns the coordinate system of my model with that of matplotlib

P01_ort = [me.dot(P01.pos_from(P0), uv) for uv in trans]        # coordinates of attachment point
P_ort   = []
DMC_ort = []
#OBS_ort = []
for i in range(1, n+1):
    P_ort.append([me.dot(P[i].pos_from(P0), uv) for uv in trans])
    DMC_ort.append([me.dot(DMC[i].pos_from(P0), uv) for uv in trans])
#    OBS_ort.append([me.dot(OBS[i-1].pos_from(P0), uv) for uv in trans])
    
pL_p = [g, m, l, r1, iXX, iYY, iZZ, c1] #, rb]
pL_p_vals = [9.8, 1., 10., 5., 1, 2, 3, 1.] #, rb1]

P01_ort_lam = sm.lambdify(qL + pL_p, P01_ort, cse=True)
P_ort_lam   = sm.lambdify(qL + pL_p, P_ort, cse=True)
DMC_ort_lam = sm.lambdify(qL + pL_p, DMC_ort, cse=True)
#OBS_ort_lam = sm.lambdify(qL + pL_p, OBS_ort, cse=True)

P_x = np.empty((schritte2, n))
P_y = np.empty((schritte2, n))
P_z = np.empty((schritte2, n))

DMC_x = np.empty((schritte2, n))
DMC_y = np.empty((schritte2, n))
DMC_z = np.empty((schritte2, n))

#OBS_x = np.empty((schritte2, n))
#OBS_y = np.empty((schritte2, n))
#OBS_z = np.empty((schritte2, n))

P01_x = np.empty(schritte2)
P01_y = np.empty(schritte2)
P01_z = np.empty(schritte2)

for i in range(schritte2):
    for j in range(n):
        P_x[i, j] = P_ort_lam(*resultat2[i, :], *pL_p_vals)[j][0]
        P_y[i, j] = P_ort_lam(*resultat2[i, :], *pL_p_vals)[j][1]
        P_z[i, j] = P_ort_lam(*resultat2[i, :], *pL_p_vals)[j][2]
        
        DMC_x[i, j] = DMC_ort_lam(*resultat2[i, :], *pL_p_vals)[j][0]
        DMC_y[i, j] = DMC_ort_lam(*resultat2[i, :], *pL_p_vals)[j][1]
        DMC_z[i, j] = DMC_ort_lam(*resultat2[i, :], *pL_p_vals)[j][2]
        
#        OBS_x[i, j] = OBS_ort_lam(*resultat2[i, :], *pL_p_vals)[j][0]
#        OBS_y[i, j] = OBS_ort_lam(*resultat2[i, :], *pL_p_vals)[j][1]
#        OBS_z[i, j] = OBS_ort_lam(*resultat2[i, :], *pL_p_vals)[j][2] 

    P01_x[i] = P01_ort_lam(*resultat2[i, :], *pL_p_vals)[0]
    P01_y[i] = P01_ort_lam(*resultat2[i, :], *pL_p_vals)[1]
    P01_z[i] = P01_ort_lam(*resultat2[i, :], *pL_p_vals)[2]


x_min = min(np.min(P_x), np.min(DMC_x), np.min(P01_x))
y_min = min(np.min(P_y), np.min(DMC_y), np.min(P01_y))
z_min = min(np.min(P_z), np.min(DMC_z), np.min(P01_z))

x_max = max(np.max(P_x), np.max(DMC_x), np.max(P01_x))
y_max = max(np.max(P_y), np.max(DMC_y), np.max(P01_y))
z_max = max(np.max(P_z), np.max(DMC_z), np.max(P01_z))

farben = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']   # I need a color for each pendulum body

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim([x_min - 1, x_max + 1])  # Adjust the limits based on your data
ax.set_ylim([y_min - 1, y_max + 1])
ax.set_zlim([z_min - 1, z_max + 1])
ax.set_xlabel('X', fontsize = 15)
ax.set_ylabel('Y', fontsize = 15)
ax.set_zlabel('Z', fontsize = 15)
ax.set_aspect('equal')

ebene = plot_3d_plane(x_min-1, x_max+1, y_min-1, y_max+1)           # plot base plane, at Z = 0.

# Initialize points and rods
LINE_P = ['P' + str(i) for i in range(1, n+1)]
LINE_DMC = ['DMC' + str(i) for i in range(1, n+1)]
#LINE_OBS = ['OBS' + str(i) for i in range(1, n+1)]
LINE_rod = ['rod' + str(i) for i in range(1, n+1)]

LINE_P0 = ax.plot([], [], [], 'o', color=farben[2], markersize=8)
for i in range(n):
    LINE_P[i], = ax.plot([], [], [], 'o', color=farben[0], markersize=3)
    LINE_DMC[i], = ax.plot([], [], [], 'o', color=farben[i], markersize=25)
#    LINE_OBS[i], = ax.plot([], [], [], 'o', color=farben[i], markersize=5)
    LINE_rod[i], = ax.plot([], [], [], '-', color=farben[0], linewidth=1)
LINE_trace, = ax.plot([], [], [], '-', color=farben[2], linewidth=0.5)          # traces the movement of the suspension point P01 of the pendulum

# Update function for animation
def update(frame):

    message = (f'Vertical Pendulum \n Running time {times2[frame]:.2f} sec')
    ax.set_title(message, fontsize=15)

    for i in range(n):
        LINE_P[i].set_data([P_x[frame, i]], [P_y[frame, i]])
        LINE_P[i].set_3d_properties([P_z[frame, i]])

        LINE_DMC[i].set_data([DMC_x[frame, i]], [DMC_y[frame, i]])
        LINE_DMC[i].set_3d_properties([DMC_z[frame, i]])

#        LINE_OBS[i].set_data([OBS_x[frame, i]], [OBS_y[frame, i]])
#        LINE_OBS[i].set_3d_properties([OBS_z[frame, i]])
    
    LINE_rod[0].set_data([P01_x[frame], P_x[frame, 0]],
                             [P01_y[frame], P_y[frame, 0]])
    LINE_rod[0].set_3d_properties([P01_z[frame], P_z[frame, 0]])

    for i in range(1, n):
        LINE_rod[i].set_data([P_x[frame, i-1], P_x[frame, i]],
                             [P_y[frame, i-1], P_y[frame, i]])
        LINE_rod[i].set_3d_properties([P_z[frame, i-1], P_z[frame, i]])
    
    LINE_P0[0].set_data([P01_x[frame]], [P01_y[frame]])
    LINE_P0[0].set_3d_properties([P01_z[frame]])

    LINE_trace.set_data(P01_x[:frame], P01_y[:frame])
    LINE_trace.set_3d_properties(P01_z[:frame])

#    for i in range(n):
#        kugel(ax, rb1, [DMC_x[frame, i], DMC_y[frame, i], DMC_z[frame, i]], farben[i])

    return LINE_P + LINE_rod + LINE_P0 + LINE_DMC + [LINE_trace]

# Create the animation
num_frames = schritte2
animation = FuncAnimation(fig, update, frames=num_frames, interval=50, blit=True)
plt.close(fig)                                                                                      # avoids the 'second' picture.
# Display the animation
HTML(animation.to_jshtml())